# Calibration backend

In [31]:
import numpy as np
import pandas as pd
from scipy.optimize import curve_fit

def calibrate(axis, voltage):
    # Check if axis is a string
    if not isinstance(axis, str):
        raise ValueError("Axis must be a string (x, y, or z)")
    
    # Read the data from the CSV file and skip every other row
    df = pd.read_csv('stage_data_final.csv')

    # Define the function for a linear regression line
    def linear_regression(v, a, b):
        return a * v + b
    
    params, covariance = curve_fit(linear_regression, df['volts (V)'], df[f'actual_distance_{axis}(um)'])

    # Errors in slope and y-intercept
    errors = np.sqrt(np.diag(covariance))
    
    calibrated_value = params[0]*voltage + params[1]
    error_calibrated_value = np.sqrt(errors[0]*voltage + errors[1])
    
    print(f'At the {voltage} V, the calibrated value with uncertainty for {axis} is {calibrated_value:.4f} ± {error_calibrated_value:.4f} µm.')
    return None

## Output

In [32]:
calibrate('x', 50)

At the 50 V, the calibrated value with uncertainty for x is 56.8627 ± 0.8097 µm.
